In [1]:
# pip install bottle
# pip install cheroot

In [2]:
# import libs
import argparse
import json
from pathlib import Path
from os import path, makedirs, getcwd
import sqlite3

from aiohttp import web
import bottle
from bottle import Bottle

In [3]:
#read cli options
parser = argparse.ArgumentParser(description='Start remote desktop server.')
parser.add_argument('--configuration', type=str, help='path of the configuration file', default='./conf/conf.json')
args, unknown = parser.parse_known_args()
#print(parser.format_help())
#print(args.configuration)
conf_path = args.configuration

In [4]:
# read configuration
if path.exists(conf_path):
    f = open(conf_path, 'r')
    conf = json.loads(f.read())
    f.close()
else:
    conf = json.loads('{}')

In [5]:
# handle absolute or relative paths in conf
def parse_path(src):
    if path.isabs(src) == False:
        src = path.join(getcwd(), path.dirname(conf_path), src)
    return path.normpath(src)

In [6]:
# read database place
if 'db' not in conf:
    conf['db'] = 'database.db'
conf['db'] = parse_path(conf['db'])

# check folder
db_folder = path.dirname(conf['db'])
if path.exists(db_folder) == False and path.dirname(db_folder) != '':
    makedirs(db_folder)

# create database
con = sqlite3.connect(conf['db'])
cur = con.cursor()

# create users
cur.execute('''
CREATE TABLE IF NOT EXISTS `users` (
    `user_id` INT(11) NOT NULL,
    `email` VARCHAR(255) NULL,
    `username` VARCHAR(255) NULL,
    `password` VARCHAR(255) NULL,
    PRIMARY KEY(`user_id`)
);
''')

# create user_sessions
cur.execute('''
CREATE TABLE IF NOT EXISTS `user_sessions` (
    `session_id` INT(11) NOT NULL,
    `user_id` INT(11) NOT NULL,
    `last_login` TIMESTAMP NULL ,
    `last_ip` VARCHAR(255) NULL,
    `expire` TIMESTAMP NOT NULL,
    `is_permanent` TINYINT(1) NOT NULL,
    PRIMARY KEY(`session_id`),
    FOREIGN KEY(`user_id`) REFERENCES `users`(`user_id`) ON UPDATE CASCADE ON DELETE CASCADE
);
''')


# create rooms
cur.execute('''
CREATE TABLE IF NOT EXISTS `rooms` (
    `room_id` INT(11) NOT NULL,
    `name` VARCHAR(255) NOT NULL,
    `is_open` TINYINT(1) NOT NULL,
    PRIMARY KEY(`room_id`)
);
''')

# create room_permissions
cur.execute('''
CREATE TABLE IF NOT EXISTS `room_permissions` (
    `permission_id` int(11) NOT NULL,
    `room_id` INT(11) NOT NULL,
    `user_id` INT(11) NOT NULL,
    `permission` TINYINT(1) NOT NULL,
    PRIMARY KEY(`permission_id`)
    FOREIGN KEY(`user_id`) REFERENCES `users`(`user_id`) ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY(`room_id`) REFERENCES `rooms`(`room_id`) ON UPDATE CASCADE ON DELETE CASCADE
);
''')

con.close()


In [7]:



if 'key' not in conf or 'cert' not in conf:
    keyfile = None
    certfile = None
else:
    conf['key'] = parse_path(conf['key'])
    conf['cert'] = parse_path(conf['cert'])
    if path.isfile(conf['key']) and path.isfile(conf['cert']):
        keyfile = conf['key']
        certfile = conf['cert']
    else:
        keyfile = None
        certfile = None

In [ ]:
from bottle import ServerAdapter, run

class SSLCherootAdapter(ServerAdapter):
    def run(self, handler):
        from cheroot import wsgi
        from cheroot.ssl.builtin import BuiltinSSLAdapter
        import ssl

        server = wsgi.Server((self.host, self.port), handler)
        server.ssl_adapter = BuiltinSSLAdapter(certfile, keyfile)

        # By default, the server will allow negotiations with extremely old protocols
        # that are susceptible to attacks, so we only allow TLSv1.2
        server.ssl_adapter.context.options |= ssl.OP_NO_TLSv1
        server.ssl_adapter.context.options |= ssl.OP_NO_TLSv1_1

        try:
            server.start()
        finally:
            server.stop()
            
base = Bottle()
@base.route("<url:re:.+>")
def index(url):
    return 'Hello world' + url

run(app=base, host='localhost', port=443, server=SSLCherootAdapter)


Bottle v0.12.25 server starting up (using SSLCherootAdapter())...
Listening on http://localhost:443/
Hit Ctrl-C to quit.

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_14352\2736852238.py:14: DeprecationWarning: ssl.OP_NO_SSL*/ssl.OP_NO_TLS* options are deprecated
  server.ssl_adapter.context.options |= ssl.OP_NO_TLSv1
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_14352\2736852238.py:15: DeprecationWarning: ssl.OP_NO_SSL*/ssl.OP_NO_TLS* options are deprecated
  server.ssl_adapter.context.options |= ssl.OP_NO_TLSv1_1


In [ ]:
try:
    # Main code
    i = 0
    while True:
        i += 1
except KeyboardInterrupt:
    # Cleanup/exiting code
    print('done!' + str(i))
